<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/training/tworzenie_modelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# zapis z csv do df
df = pd.read_csv('file_name.csv')

# podgląd danych
df.head()

# statystyki danych numerycznych
df.describe().apply(lambda x: round(x, 2))

# statystyki danych tekstowych. Można też include="category"
df.describe(include="object")

# samo df.isnull() zwraca w tabeli True tam gdzie jest brak danych
df.isnull().sum()

# takie kolumny warto zostawić
selected_vars = [var1, var4, ...]

# z df wycina kolumny spośród selected_vars
df_selected = df[selected_vars]

# oddziela dane od targetu, gdy target jest ostatnią koumną
X = df_selected[df_selected.columns[:-1]]
# pobiera ostatnią kolumnę target jako Series
y = df_selected[df_selected.columns[-1]]


# oddziela dane od targetu, gdy target nie jest ostatnią kolumną
# Lista nazw wszystkich kolumn
all_columns_names = list(df.columns)
print(all_columns_names)
# Lista nazw kolumn z atrybutami
data_columns_names = [column_name for column_name in all_columns_names if column_name != "target_name"]
print(data_columns_names)
# df zawierająca kolumny z atrybutami
X = df[data_columns_names]
print(X)
# wydobywa target jako Series, gdy nie jest on ostatnią kolumną
y = df["target_name"]
print(y)


# Zamiana wartości tekstowych na numeryczne (bo sklearn tylko liczby przyjmuje do modeli).
# Parametr drop_first=True usuwa pierwszą kolumnę, którą jest "no" i zostawia "yes"
# Można też z sklearn OneHotEncoding
X = pd.get_dummies(X, drop_first=True)

from sklearn.model_selection import train_test_split
# dzieli dane. Można stratify=tagret
# domyślnie X_test zajmuje 25% całego X (można zmienić np test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# skalowanie cech numerycznych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# dopasowuję tylko do danych treningowych, żeby zapobiec wyciekowi danych ze zbioru testowego
scaler.fit(X_train)

# ale już transformuje dane treningowe i testowe
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LogisticRegression
# obiekt klasyfikatora lub regresora
# na przykładzie regresji logistycznej, która jest tak naprawdę klasyfikatorem
log_reg = LogisticRegression()

# nauczenie modelu
log_reg.fit(X_train, y_train)

# klasyfikatory mają score accuracy a regresory R2 (odmiana MSE)
log_reg.score(X_test, y_test)

from sklearn.model_selection import GridSearchCV

# przykładowa lista słowników (tu jeden słownik więc nie trzeba []) dla drzewa decyzyjnego
params = {'max_depth': np.arange(1, 10),
         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]}

grid_search = GridSearchCV(classifier, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# najlepszy estymator, który bierzemy jako model
grid_search.best_estimator_

# najepsze parametry estymatora
grid_search.best_params_

# wykres granic decyzyjnych
from mlxtend.plotting import plot_decision_regions

plt.figure(figsize=(10, 8))
plot_decision_regions(X_test, y_test, grid_search)
plt.title(f'Zbiór treningowy: dokładność {grid_search.score(X_train, y_train):.4f}')
plt.show()

# metryka dokładkość (dla klasyfikatorów)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

# dokładność na zbiorze testowym
grid_search.score(X_test, y_test)


# macierz pomyłek jako macierz numpy
from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm


# macierz pomyłek jako heatmapa (mapa ciepła) w seaborn
plt.figure(figsize=(8, 6))
plt.title('Macierz konfuzji')
_ = sns.heatmap(cm, annot=True, cmap=sns.cm.rocket_r)


# macierz pomyłek jako heatmapa (mapa ciepła) w plotly (interaktywna)
import plotly.figure_factory as ff

# w miejsce n wpisać ilość klas 
amount_of_classes = n

columns = ['pred_' + str(i) for i in range(amount_of_classes)]
index = ['true_' + str(i) for i in range(amount_of_classes)]

def plot_confusion_matrix(cm):
    # Mulitclass classification, n classes
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=columns, index=index[::-1])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), 
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=700, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)